In [1]:
import pandas as pd
import sqlite3
import seaborn as sb
import matplotlib as plt

In [2]:
conn = sqlite3.connect('space_missions.sqlite')
cur = conn.cursor()

conn2 = sqlite3.connect('space_missions_insights.sqlite') 
cur = conn.cursor()

In [3]:
# With the obtained previous data of launches, we query top 5 countries with successful launches to obtain the launch location
q = '''
SELECT Company, Country, Location, Date, Rocket, Mission
FROM records
WHERE (Country = 'USA' OR Country = 'Russia' OR Country = 'Kazakhstan' OR Country = 'China' OR Country = 'France');
'''
df = pd.read_sql_query(q,conn)
df.to_sql('top5_launc_loc', conn2, if_exists='replace', index=False)

4285

In [4]:
q2 ='''
SELECT SUBSTR(Date, -4) AS año, COUNT(*) AS launches_per_year
FROM records
GROUP BY SUBSTR(Date, -4)
ORDER BY launches_per_year DESC
LIMIT 5;
'''

df2 = pd.read_sql_query(q2,conn)
df.to_sql('launches_per_year', conn2, if_exists='replace', index=False)

4285

In [5]:
# Determining the active & retired rockets
q3= '''
SELECT Country, 
       SUM(CASE WHEN RocketStatus = 'Active' THEN 1 ELSE 0 END) AS Active,
       SUM(CASE WHEN RocketStatus = 'Retired' THEN 1 ELSE 0 END) AS Retired
FROM records
GROUP BY Country
ORDER BY Country;
'''
df3 = pd.read_sql_query(q3, conn)
df3.to_sql('Act_Ret_rockets', conn2, if_exists='replace', index=False)

22

In [6]:
# A filter with rocket info

q4 = '''
SELECT Country, Company, SUBSTR(Date, -4) AS Year, Rocket, MissionStatus, RocketStatus
FROM records
'''
df4 = pd.read_sql_query(q4, conn)
df4.to_sql('gral_rockt_info', conn2, if_exists='replace',index=False)

4630